In [1]:
import pandas as pd
import os, time, logging
import numpy as np
from src.data import Hicmat, plot_data, preprocess_data
from src.tad_algo import TopDom, TADtree
from src.utils import *
import CTCF

In [2]:
def bedPicks(file, chrom, resolution):
    
    df = pd.read_csv(file, sep='\t', comment = 't', header=None)
    header = ['chrom', 'chStart', 'chEnd', 'name', 'score', 'strand', 'sigValue', 'pValue', 'qValue', 'peak']
    df.columns = header[:len(df.columns)]
    l_peak = []                               #to store pairs (chromStart, chromEnd) for a specific chrom
    
    #delete non useful columns 
    if set(df['name'])=={'.'}:
        del df['name']
    if set(df['strand'])=={'.'}:
        del df['strand']
    if set(df['pValue'])=={-1.0}:
        del df['pValue']
        
    #we take into account data for a specific chromosome 
    df = df[df['chrom']==chrom]
    df = df.sort_values(by = 'chStart')
    
    #just some tests for a eventual filter 
    score = np.array(df['sigValue'])
    #print(score.mean(), score.min(), score.max()) 
    index = df.index.tolist()
    for ctcf in index:
        if int(df['sigValue'][ctcf])>=140:
            l_peak.append(int(round(df['chStart'][ctcf]/resolution, 0)))
            l_peak.append(int(round(df['chEnd'][ctcf]/resolution, 0)))
    return list(set(l_peak))

def evaluate(TADs, ctcf):
    a=0
    b=[]
    c=set()
    for tad in TADs:
        if tad[0] in ctcf and tad[0] not in b:
            a+=1
            b.append(tad[0])
        if tad[1] in ctcf and tad[1] not in b:
            a+=1
            b.append(tad[1])
        c.add(tad[0])
        c.add(tad[1])
    return round(a/len(c), 4)*100

In [3]:
def get_great_tads(TADs, lim):
    TADs_set = set(TADs)
    for tad_i in range(len(TADs)-1):
        for tad_j in range(tad_i+1, len(TADs)):
            if TADs[tad_j][1]-TADs[tad_i][0]<=lim:
                TADs_set.add((TADs[tad_i][0], TADs[tad_j][1]))
            else:
                break
    return sorted(TADs_set)

def get_common_tads(list1, list2, gap):
    output = []
    for tad in list1:
        for i in range(-gap, gap+1):
            for j in range(-gap, gap+1):
                if (tad[0]+i, tad[1]+j) in list2:
                    output.append((min(tad[0], tad[0]+i), max(tad[1], tad[1]+j)))
    return output

def get_overlaps(TADs):
    counter = 0
    n = len(TADs)
    overlaps = []
    for i in range(n-1):
        for j in range(i+1, n):
            if TADs[i][0]!=TADs[j][0] and TADs[i][1]>TADs[j][0] and TADs[i][1]<TADs[j][1]:
                counter+=1
                overlaps.append((TADs[i], TADs[j]))
            elif TADs[i][1]<TADs[j][0]:
                break
    return counter, overlaps

def get_count_boundaries(TADs):
    count_boundaries = {}
    for tad in TADs:
        count_boundaries.setdefault(tad[0], 0)
        count_boundaries.setdefault(tad[1], 0)
        count_boundaries[tad[0]]+=1
        count_boundaries[tad[0]]+=1
    return count_boundaries

def get_smoothing(TADs):
    _, overlaps = get_overlaps(TADs)
    count_boundaries = get_count_boundaries(TADs)
    new_boundaries = {}
    for tads in overlaps:
        if count_boundaries[tads[0][1]]>count_boundaries[tads[1][0]] and tads[1][0] not in new_boundaries:
            new_boundaries[tads[1][0]]=tads[0][1]
        elif tads[0][1] not in new_boundaries:
            new_boundaries[tads[0][1]]=tads[1][0]
    output = []
    for tad in TADs:
        if tad[0] in new_boundaries:
            if tad[1] in new_boundaries:
                output.append((new_boundaries[tad[0]], new_boundaries[tad[1]]))
            else:
                output.append((new_boundaries[tad[0]], tad[1]))
        else:
            if tad[1] in new_boundaries:
                output.append((tad[0], new_boundaries[tad[1]]))
            else:
                output.append(tad)
    return sorted(set(output))

def consensus2(lists, resolution, gap=200000, lim=3000000, smoothing=True):
    lim = int(lim/resolution)
    extended_lists = []
    for list_i in lists:
        extended_lists.append(get_great_tads(list_i, lim))
    gap = int(gap/resolution)
    output = []
    for i in range(len(extended_lists)-1):
        for j in range(i+1, len(extended_lists)):
            output+=get_common_tads(extended_lists[i], extended_lists[j], gap)
    output = sorted(set(output))
    if smoothing:
        output = get_smoothing(output)
    return output

In [4]:
def smoothing_with_ctcf(true_TADs, ctcf):
    counter=0
    n=len(true_TADs)
    for i in range(n-1):
        for j in range(i+1, n):
            if true_TADs[i][0]<true_TADs[j][0] and true_TADs[i][1]<true_TADs[j][1] and true_TADs[i][1]>true_TADs[j][0]:
                middle = int((true_TADs[i][1]-true_TADs[j][0])/2)
                for boundary in range(true_TADs[j][0], true_TADs[i][1]+1):
                    if boundary in ctcf:
                        true_boundary = boundary
                        counter+=1
                        break
                    else:
                        true_boundary = true_TADs[j][0]+middle
                true_TADs[i] = (true_TADs[i][0], true_boundary)
                true_TADs[j] = (true_boundary, true_TADs[j][1])
            if true_TADs[i][0]>true_TADs[j][0] and true_TADs[i][1]>true_TADs[j][1] and true_TADs[i][0]<true_TADs[j][1]:
                middle = int((true_TADs[j][1]-true_TADs[i][0])/2)
                for boundary in range(true_TADs[j][0], true_TADs[i][1]+1):
                    if boundary in ctcf:
                        true_boundary = boundary
                        counter+=1
                        break
                    else:
                        true_boundary = true_TADs[i][0]+middle
                true_TADs[i] = (true_boundary, true_TADs[i][1])
                true_TADs[j] = (true_TADs[j][0], true_boundary)
    print(counter)
    return true_TADs

In [5]:
fileCTCF = os.path.join('..', 'CTCF', 'ENCFF796WRU.bed')
l_peak_GM = bedPicks(fileCTCF, 'chr1', 100000)
true_results = read_arrowhead_result(os.path.join('..', 'RESULTS', 'GSE63525_GM12878_primary+replicate_Arrowhead_domainlist.txt'), '1', 100000)

In [6]:
tads_by_tadtree = pd.read_csv('../CHROMOSOMES/GM12878/100kb/TADtree_outputs/chr1/N399.txt', delimiter='\t')
tads_by_tadtree = tads_by_tadtree.iloc[:, [1,2]]
tadtree = []
for i in range(len(tads_by_tadtree['start'])):
    tadtree.append((tads_by_tadtree['start'][i], tads_by_tadtree['end'][i]))
#print(tadtree)

In [7]:
folder = os.path.join('..', 'CHROMOSOMES', 'GM12878', '100kb')
data_path = os.path.join(folder, 'chr1_100kb.npy')
resolution=100000
if not os.path.isfile(data_path):
    preprocess_data(folder, resolution)
hic_mat = Hicmat(data_path, resolution)
hic_mat.filter(threshold = 1)
topdom = TopDom()
topdom = topdom.getTADs(hic_mat, window=10)
for i in range(len(topdom)):
    topdom[i]=(int(topdom[i][0]/resolution), int(topdom[i][1]/resolution))
#print('\n', topdom)

TopDom Step 1 : Generating binSignals by computing bin-level contact frequencies
TopDom Step 2 : Detect TD boundaries based on binSignals
TopDom Step 3 : Statistical Filtering of false positive TD boundaries
TopDom : Exporting TADs


## Average sizes of TADs

In [8]:
print('TopDom :', np.mean([tad[1]-tad[0] for tad in topdom]))
print('TadTread :', np.mean([tad[1]-tad[0] for tad in tadtree]))
print('ArrowHead :', np.mean([tad[1]-tad[0] for tad in true_results]))

TopDom : 21.939393939393938
TadTread : 6.077540106951871
ArrowHead : 2.404949381327334


## Tests of the consensus without smoothing

In [9]:
tadtree_topdom = consensus2([tadtree, topdom], 100000, smoothing=False)
tadtree_arrowhead = consensus2([tadtree, true_results], 100000, smoothing=False)
topdom_arrowhead = consensus2([topdom, true_results], 100000, smoothing=False)
tadtree_topdom_arrowhead = consensus2([tadtree, topdom, true_results], 100000, smoothing=False)

### Number of overlaps

In [10]:
print(get_overlaps(tadtree)[0], get_overlaps(topdom)[0], get_overlaps(true_results)[0])
print(get_overlaps(tadtree_topdom)[0], get_overlaps(tadtree_arrowhead)[0], get_overlaps(topdom_arrowhead)[0])
print(get_overlaps(tadtree_topdom_arrowhead)[0])

0 0 0
38 588 0
694


### Comparison with the CTCF peaks

In [11]:
print('Rate of similarity')
print('TadTree TopDom Arrowhead')
print(evaluate(tadtree, l_peak_GM),'% ', evaluate(topdom, l_peak_GM),'% ', evaluate(true_results, l_peak_GM), '%')
print('The three consensus of two methods')
print(evaluate(tadtree_topdom, l_peak_GM),'% ', evaluate(tadtree_arrowhead, l_peak_GM),'% ', evaluate(topdom_arrowhead, l_peak_GM),'%')
print('The consensus of the three methods')
print(evaluate(tadtree_topdom_arrowhead, l_peak_GM),'%')

Rate of similarity
TadTree TopDom Arrowhead
61.01 %  47.17 %  61.480000000000004 %
The three consensus of two methods
57.69 %  63.93 %  50.0 %
The consensus of the three methods
63.28 %


## Tests of the consensus with smoothing

In [12]:
tadtree_topdom = consensus2([tadtree, topdom], 100000)
tadtree_arrowhead = consensus2([tadtree, true_results], 100000)
topdom_arrowhead = consensus2([topdom, true_results], 100000)
tadtree_topdom_arrowhead = consensus2([tadtree, topdom, true_results], 100000)

### Number of overlaps

In [13]:
print(get_overlaps(tadtree)[0], get_overlaps(topdom)[0], get_overlaps(true_results)[0])
print(get_overlaps(tadtree_topdom)[0], get_overlaps(tadtree_arrowhead)[0], get_overlaps(topdom_arrowhead)[0])
print(get_overlaps(tadtree_topdom_arrowhead)[0])

0 0 0
0 24 0
56


### Comparison with the CTCF peaks

In [14]:
print('Rate of similarity')
print('TadTree TopDom Arrowhead')
print(evaluate(tadtree, l_peak_GM),'% ', evaluate(topdom, l_peak_GM),'% ', evaluate(true_results, l_peak_GM), '%')
print('The three consensus of two methods')
print(evaluate(tadtree_topdom, l_peak_GM),'% ', evaluate(tadtree_arrowhead, l_peak_GM),'% ', evaluate(topdom_arrowhead, l_peak_GM),'%')
print('The consensus of the three methods')
print(evaluate(tadtree_topdom_arrowhead, l_peak_GM),'%')

Rate of similarity
TadTree TopDom Arrowhead
61.01 %  47.17 %  61.480000000000004 %
The three consensus of two methods
56.25 %  62.91 %  50.0 %
The consensus of the three methods
61.95 %
